# TP5 - Inferencia Bayesiana

Considere el modelo Beta-Binomial para la moneda.

1. Implementacion
Implemente el modelo en pymc3 para los priors Beta(1,1) y Beta(100,100), y grafique las posteriors resultantes de observar 6 caras de 6 tiradas.

2. Resúmenes
Calcule el MAP (maximo a posteriori) y las medias de ambas posteriors.

3. Prediccion
Prediga, para ambos priors, el resultado de una nueva tirada de moneda.

4. Otros datos
Repita los puntos anteriores para una situacion en la que se observan 20 caras de 20 tiradas, y otra en la que se observan 3 caras de 3 tiradas. ¿Qu ́e puede concluir de los priors elegidos? ¿Y del modelo en general?

5. Prior 
Pensando en su propia experiencia, considere cu ́antas caras consecutivas le har ́ıan empezar a sospechar que la moneda está cargada. A partir de este número y usando el modelo, diga cuáles son los parámetros de su prior Beta.

http://people.duke.edu/~ccc14/sta-663-2016/16C_PyMC3.html

In [1]:
import pymc3
import numpy.random as rng
import scipy.stats as stats

ImportError: No module named 'pymc3'